### background
I have a list of N people who need to be seated in a boat in N seats. 
Boat has one cox, and 7 pairs of rowers. N = 15.
People need to be matched by weight, such that the boat is balanced in terms of roll and pitch (https://commons.wikimedia.org/wiki/File:Rotations.png#/media/File:Rotations.png)
and by height, so that people have neighbours of similar heights (to make rowing as a unit easier).
People also have preferences:
* To cox or not to cox. 
* Sit in the front/back/middle of the boat. NB: height makes it uncomfortable for tall people to sit in the first or last row.
* In case of switching, some people would like to switch sides.

Switching.
in total there are 4 seating plans to make, since seating will be switched 4 times throughout the race (see last point in preferences).

### solution
it can be seen as a variant of a stable-marriage problem (https://en.wikipedia.org/wiki/Stable_marriage_problem). SMP is the problem of finding a stable matching between two equally sized sets of elements given an ordering of preferences for each element. 
There are two differences: 
* it is unidirectional (so rowers *prefer* sits, but sits *do not prefer* rowers).
* it is with indifference, so it needs not to exist a strict order of preferences, e.g. top 3 sits are given, rest is of equal priority. see https://en.wikipedia.org/wiki/Stable_marriage_with_indifference

the assignment problem might be good to look into, if we are able to translate weight/height matches into some edge weights. The assignment problem consists of finding, in a weighted bipartite graph, a matching in which the sum of weights of the edges is as large as possible. A common variant consists of finding a minimum-weight perfect matching. see https://en.wikipedia.org/wiki/Assignment_problem

#### for NetPharMed:
There is an implementation of a classical bidirectional aglo implemented as Matcher() below. It matches rowers with seats. From the point of view of a rower, it is truthful, ie no better seating can be achieved by misinforming the preferences. To adapt it for office seating, make all the sits have identical preferences for rowers. That can also be used in the git manual, to make people branch the file, input their preferences, and then submit a pull request.

#### for Sulkava:
how to make matching by weight/height happen? One idea is to use simulated annealing. SA is an optimization technique to find a global optimum of a function, esp when the search space is discrete. For problems where finding an *approximate global optimum* is more important than finding a *precise local optimum in a fixed amount of time*, simulated annealing may be preferable to alternatives such as gradient descent. The notion of slow cooling (from metallurgy) implemented in the simulated annealing algorithm is interpreted as a slow decrease in the probability of accepting worse solutions as the solution space is explored. At each time step, the algorithm randomly selects a solution close to the current one, measures its quality, and then decides to move to it or to stay with the current solution based on either one of two probabilities between which it chooses on the basis of the fact that the new solution is better or worse than the current one. During the search, the temperature is progressively decreased from an initial positive value to zero and affects the two probabilities: at each step, the probability of moving to a better new solution is either kept to 1 or is changed towards a positive value; on the other hand, the probability of moving to a worse new solution is progressively changed towards zero. Here it is in gif form: https://commons.wikimedia.org/wiki/File:Hill_Climbing_with_Simulated_Annealing.gif#/media/File:Hill_Climbing_with_Simulated_Annealing.gif

Here is the implementation of SA for wedding table seating (where there are N tables for P people, each table sits M people, match people's preferences so that they do not sit with those they hate, and tend to preserve their groups of friends. https://lukeplant.me.uk/blog/posts/wedding-hacks---seating-planner-using-simulated-annealing/ The idea is that you define a matrix of connections, with a zero indicating that the people don’t know each other, 1 that they do know each other, plus 50 indicating that they must be together, and negative numbers for people who should be kept apart!
Here is an alternative to it: http://linanqiu.github.io/2018/03/05/Wedding-Seat-Optimization/ with a Jup notebook https://github.com/linanqiu/wedding-optimization-simulated-annealing/blob/master/wedding-optimization.ipynb
and another https://github.com/zneedell/seating_optimization/blob/master/main.ipynb

Here is a different solution of the wedding seat solved using hypergraphs. We model it as a hypergraph partitioning problem and solving it with KaHyPar (http://kahypar.org/ / http://github.com/SebastianSchlag/kahypar). Every guest is represented by a node and weighted hyperedges express relationships (a couple and their kids, extended family, friends, acquaintances, etc). The goal is to partition the hypergraph into (roughly) equal-sized parts (i.e., tables) while minimising the sum of weights of cut hyperedges ("λ-1 metric").

This is a solution using simple linear programming https://stackoverflow.com/questions/19143474/which-algorithm-could-solve-my-wedding-table-issue but without code.

[16:45, 7/3/2019] Tiia: Ok 😂  you can ignore preferences if they clash with measurements. I'd say tall to short from cox to back, at least I kept hitting Elli's oar when I rowed behind her
[16:46, 7/3/2019] Tiia: And Karen should be close to front / Cox, cause she has a hearing problem

In [1]:
%pylab inline
import pandas as pd
import numpy as np
from collections import defaultdict, OrderedDict

Populating the interactive namespace from numpy and matplotlib


In [2]:
# refer to gale-shapley 1962 paper for the original algo


class Matcher:

    def __init__(self, rowers, sits, forbidden):
        '''
        Constructs a Matcher instance.
        Takes a dict of rowers's sittings preferences, `rowers`,
        a dict of special sits to be allocated e.g. white oars, `sits`. For now it is identical for all the rowers
        and a dict specifying which sittings are forbidden
        for each rower:
        >>> forbidden = { 'name': ['3', '4', ... ] }
        '''
        self.R = rowers
        self.S = sits
        self.forbidden = forbidden
        self.sittings = {}
        self.pairs = []

        # we rank sitting preferences at initialization 
        # to avoid expensive lookups when matching
        self.rrank = defaultdict(dict)  # `rrank[r][s]` is r's ranking of s
        self.srank = defaultdict(dict)  # `srank[s][r]` is s's ranking of r

        for r, prefs in rowers.items():
            for i, s in enumerate(prefs):
                self.rrank[r][s] = i

        for s, prefs in sits.items():
            for i, r in enumerate(prefs):
                self.srank[s][r] = i

    def __call__(self):
        return self.match()

    def prefers(self, s, r, h):
        '''
        Test whether s prefers r over h.
        '''
        return self.srank[s][r] < self.srank[s][h]

    def is_forbidden(self, r, s):
        '''
        Test whether (r, s) is a forbidden pairing.
        '''
        return s in self.forbidden.get(r, [])

    def after(self, r, s):
        '''
        Return the sit favored by r after s.
        
        '''
        prefs = self.R[r]               # r's ordered list of preferences
        i = self.rrank[r][s] + 1        # index of sits following s in list of prefs
        if i >= len(prefs):
            return ''                   # no more sits left!
        s = prefs[i]                    # sit following s in list of prefs
        if self.is_forbidden(r, s):     # if r, s) is forbidden
            return self.after(r, s)     # try next w 
        return s

    def match(self, rowers=None, next=None, sittings=None):
        '''
        Try to match all rowers with their next preferred sit.
        
        '''
        if rowers is None: 
            rowers = list(self.R)         # get the complete list of rowers
        if next is None: 
            # if not defined, map each rower to their first preference
            next = dict((r, rank[0]) for r, rank in self.R.items()) 
        if sittings is None: 
            sittings = {}                  # mapping from sit to current sitting
        if not len(rowers): 
            self.pairs = [(h, s) for s, h in sittings.items()]
            self.sittings = sittings
            return sittings
        r, rowers = rowers[0], rowers[1:]
        s = next[r]                     # next sit for m to choose
        if not s:                       # continue if no sit to choose
            return self.match(rowers, next, sittings)
        next[r] = self.after(r, s)      # sit after s in r's list of prefs
        if s in sittings:
            h = sittings[s]                # current sitting
            if self.prefers(s, r, h):
                rowers.append(h)           # sit becomes available again
                sittings[s] = r            # s becomes sit for r
            else:
                rowers.append(r)           # r remains unsitted
        else:
            sittings[s] = r                # s becomes s for r
        return self.match(rowers, next, sittings)

    def is_stable(self, sittings=None, verbose=False):
        if sittings is None:
            sittings = self.sittings
        for s, r in sittings.items():
            i = self.R[r].index(s)
            preferred = self.R[r][:i]
            for p in preferred:
                if p in self.forbidden.get(r, []):  # no need to worry about
                    continue                        # forbidden configs
                if not p in sittings:
                    continue
                h = sittings[p]
                if self.S[p].index(r) < self.S[p].index(h):  
                    msg = "{}'s sitting to {} is unstable: " + \
                          "{} prefers {} over {} and {} prefers " + \
                          "{} over the current sitting {}"
                    if verbose:
                        print(msg.format(r, s, r, p, s, p, r, h))
                    return False
        return True


In [3]:
# rowers is just people with their sit preferences
rowers = "tiia: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15; alison: 1, 2, 3, 4, 5, 10, 11, 12, 13, 6, 7, 8, 9, 14, 15; bulat: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15; arvydas: 1, 2, 3, 13, 14, 15, 4, 5, 6, 7, 8, 9, 10, 11, 12; julia: 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5; lauri: 1, 2, 10, 11, 12, 3, 4, 5, 6, 7, 8, 9, 13, 14, 15; tatiana: 5, 6, 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 14, 15; ulrika: 7, 8, 9, 10, 11, 12, 13, 1, 2, 3, 4, 5, 6, 14, 15; karen: 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10; hans: 11, 12, 13, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 14, 15; jaana: 9, 10, 11, 1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15; joseph: 1, 2, 11, 12, 13, 14, 15, 3, 4, 5, 6, 7, 8, 9, 10; elli: 9, 10, 11, 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8; elina: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15; kul: 12, 13, 14, 15, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11"                               
# sits is 
sits = "1: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 2: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 3: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 4: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 5: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 6: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 7: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 8: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 9: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 10: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 11: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 12: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 13: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 14: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 15: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul"

forbidden = str()
# forbidden = "1: jaana, joseph; 2: joseph, elli; 3: elli, elina; 4: elina, kul; 5: kul, tiia; 6: tiia, alison; 7: alison, bulat; 8: bulat, arvydas; 9: arvydas, julia; 10: julia, lauri; 11: lauri, tatiana; 12: tatiana, ulrika; 13: ulrika, karen; 14: karen, hans; 15: hans, jaana"

In [4]:
sits = str()
for i in range(1,16):
    sits = sits + " " + "{}: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul;".format(i)
sits = sits.rstrip(';')
print(sits)

 1: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 2: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 3: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 4: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 5: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 6: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 7: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 8: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 9: arvydas, julia, karen, tiia, bulat, elina, hans, joseph, lauri, ulrika, tatiana, jaana, alison, elli, kul; 10: arvyd

In [5]:
def dictor(s, void = False):
    '''
    returns a dict from a string. Given in the following format
    name0: preference0, preferenceN; nameK: preference0, preferenceN;
    '''
    if void:
        return dict()
    return dict((k.strip(), v.strip().replace(',','').split()) for k,v in (item.split(':') for item in s.split(';')))

In [6]:
r = dictor(rowers)
s = dictor(sits)
f = dictor(forbidden, void=True)  # as there are no forbidden sittings

In [7]:
out = Matcher(r, s, f)

In [8]:
one = out()

In [9]:
dtypes = {'cox': bool,'name': str, 'height [cm]': np.float, 'weight [kg]': np.float, 'notes': str}
prefs = pd.read_csv('prefs.csv', sep=';', index_col=False, dtype=dtypes)

# accomodate to elina's preference of sitting in front, by manually setting her height to height = height + 7
prefs.loc[prefs['name'] == 'elina', 'height [cm]'] = 172.0
### I CANNOT DO THAT IT WOULD MESS THINGS UP A GREAT DEAL
#prefs.loc[prefs['name'] == 'tiia', 'height [cm]'] = 174.1


In [45]:
prefs

,cox,name,height [cm],weight [kg],notes
0,True,tiia,166.0,55.0,"rower/white oar, can cox 1/4"
1,False,alison,164.0,64.1,Rower
2,False,bulat,174.0,79.0,NaN
3,True,arvydas,181.0,77.0,"Cox, white"
4,True,julia,175.0,70.0,I usually do worse in the last two rows becaus...
5,False,lauri,170.0,65.1,NaN
6,False,tatiana,169.0,55.5,rower or white oar if needed
7,False,ulrika,169.1,64.5,Rower or white if needed
8,True,karen,173.5,67.5,"Rower 1st/2nd row, white oar (if needed), also..."
9,False,hans,179.0,73.0,NaN


In [10]:
# create a dict object, where name: {weight, height, cox}
d = dict()
for i in range(len(prefs)):
    d[prefs.iloc[i,:]['name']] = {'height': prefs.iloc[i,:]['height [cm]'], 'weight': prefs.iloc[i,:]['weight [kg]'], 'cox': prefs.iloc[i,:]['cox']}

# prep for rankign 
heights = dict()
coxes = dict()
for k,v in d.items():
    heights[k]= v['height']
for k,v in d.items():
    coxes[k]= v['cox']
    

In [11]:
d

{'tiia': {'height': 166.0, 'weight': 55.0, 'cox': True},
 'alison': {'height': 164.0, 'weight': 64.1, 'cox': False},
 'bulat': {'height': 174.0, 'weight': 79.0, 'cox': False},
 'arvydas': {'height': 181.0, 'weight': 77.0, 'cox': True},
 'julia': {'height': 175.0, 'weight': 70.0, 'cox': True},
 'lauri': {'height': 170.0, 'weight': 65.1, 'cox': False},
 'tatiana': {'height': 169.0, 'weight': 55.5, 'cox': False},
 'ulrika': {'height': 169.1, 'weight': 64.5, 'cox': False},
 'karen': {'height': 173.5, 'weight': 67.5, 'cox': True},
 'hans': {'height': 179.0, 'weight': 73.0, 'cox': False},
 'jaana': {'height': 167.0, 'weight': 62.0, 'cox': False},
 'joseph': {'height': 170.1, 'weight': 65.0, 'cox': False},
 'elli': {'height': 163.0, 'weight': 60.0, 'cox': False},
 'elina': {'height': 172.0, 'weight': 55.1, 'cox': False},
 'kul': {'height': 157.0, 'weight': 64.0, 'cox': False}}

In [12]:
# ranking by height https://stackoverflow.com/questions/30282600/python-ranking-dictionary-return-rank

heights_rank = {key: rank for rank, key in enumerate(sorted(heights, key=heights.get, reverse=True), 1)}

# hack to accomodate for tiia wanting to cox (tiia ~ hans) 
# ToDo: for future use cox [True, False]
temp = heights_rank['hans']
heights_rank['hans'] = heights_rank['tiia']
heights_rank['tiia'] = temp



for k, v in heights_rank.items():
    d[k]['rank_by_height'] = v  # do not modify in-place

In [13]:
dd = d.copy()

In [14]:
rng = np.random.RandomState(1234) # for reproducibility

for k, v in dd.items():
    out = list()
    if v['rank_by_height'] == 1: # for highest rank it is just range(1,15)
        rest = list(range(v['rank_by_height'],len(dd)+1))
        out = out + rest
    elif v['rank_by_height'] == len(dd): # for lowest rank it is arange(15,1,-1)
        rest = list(arange(v['rank_by_height'],0,-1))
        out = out + rest
    elif v['rank_by_height'] == 2: #special handling for tiia, otherwise she gets rank 0
        rest = [v['rank_by_height'],v['rank_by_height']+1,v['rank_by_height']-1,v['rank_by_height']+2]
        out = rest + list(rng.permutation(list(set(set(np.arange(1,16,1))).difference(rest))))
    elif v['rank_by_height'] < len(dd): # check that it is less than 15, then start building rest by +/- 1
        rest = [v['rank_by_height'],v['rank_by_height']+1,v['rank_by_height']-1]
        if v['rank_by_height']+1 < len(dd):
            rest = rest + [v['rank_by_height']+2,v['rank_by_height']-2]
        out = rest + list(rng.permutation(list(set(set(np.arange(1,16,1))).difference(rest))))  # here we get elements from range(1,15) that are not in rest, and then we shuffle them. list(set(set)) is needed because set returns ordered ??? values
    v['rank_by_height_full'] = out

#    dd[k]['rank_height_full'] = v['rank_height']


In [15]:
rr = dict()
for k,v in dd.items():
    rr[k] = v['rank_by_height_full']

In [16]:
out1 = Matcher(rr, s, f)

In [17]:
two = out1()

#### by comparing one and two we should see a difference. Random layout to actually correct one is achieved using OrderedDict

### mind the fact that one has commas around keys, another doesnt. Check when identical

In [18]:
two_w_weights = OrderedDict()
# trying to add the weights
for k,v in two.items():
    two_w_weights[k] = {'name': v, 'weight': dd[v]['weight']}
    
two_w_weights = OrderedDict(sorted(two_w_weights.items()))

In [35]:
cox, left_side, right_side = [], [], []  # let's take 2,4 ..., as left side and 3,5 ... as right
for k,v in two_w_weights.items():
    if k == 1:  # no need to put in the cox. Cox always has the rank of 1
        cox.append([v['name'], k, v['weight']])
    elif k % 2 == 0:
        left_side.append([v['name'], k, v['weight']])
    else:
        right_side.append([v['name'], k, v['weight']])

### here we are adding weights to each sides. Unoptimized

In [36]:
left_sum, right_sum = 0,0
for i in zip(left_side, right_side): # so i[0] is left and i[1] is right
    if left_sum == right_sum:
        left_sum += i[0][2]
        right_sum += i[1][2]
    else:
        left_sum += i[0][2]
        right_sum += i[1][2]

print(left_sum, right_sum)

442.7 457.1


### here we are adding weights to each sides. Optimized

In [37]:
left_side_opt, right_side_opt = [], []
left_sum, right_sum = 0,0
for i in zip(left_side, right_side): # so i[0] is left and i[1] is right
    if left_sum == right_sum:
        left_sum += i[0][2]
        left_side_opt.append(i[0])
        right_sum += i[1][2]
        right_side_opt.append(i[1])
    else:
        if left_sum >= right_sum: # add smaller to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[1][2]
                left_side_opt.append(i[1])
                right_sum += i[0][2]
                right_side_opt.append(i[0])
            else: 
                left_sum += i[0][2]
                left_side_opt.append(i[0])
                right_sum += i[1][2]
                right_side_opt.append(i[1])
        else:  # add bigger to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[0][2]
                left_side_opt.append(i[0])
                right_sum += i[1][2]
                right_side_opt.append(i[1])
            else: 
                left_sum += i[1][2]
                left_side_opt.append(i[1])
                right_sum += i[0][2]
                right_side_opt.append(i[0])
                
print(left_sum, right_sum)


452.0 447.79999999999995


In [38]:
# first sitting

to_write = pd.DataFrame([cox, [i[0] for i in left_side_opt], [i[0] for i in right_side_opt]])
to_write.index = ['cox', 'left_side', 'right_side']
to_write.to_csv('sitting1.csv', header=False)



In [39]:
left_side_opt, right_side_opt

([['tiia', 2, 55.0],
  ['bulat', 4, 79.0],
  ['joseph', 7, 65.0],
  ['ulrika', 9, 64.5],
  ['tatiana', 10, 55.5],
  ['hans', 12, 73.0],
  ['elli', 14, 60.0]],
 [['julia', 3, 70.0],
  ['karen', 5, 67.5],
  ['elina', 6, 55.1],
  ['lauri', 8, 65.1],
  ['jaana', 11, 62.0],
  ['alison', 13, 64.1],
  ['kul', 15, 64.0]])

In [40]:
# second sitting. Cox becomes tiia

temp = left_side_opt[0]
left_side_opt[0] = right_side_opt[0]
right_side_opt[0] = cox[0]
cox = [temp]




In [41]:


left_side_opt1, right_side_opt1 = [], []
left_sum, right_sum = 0,0

for i in zip(left_side_opt, right_side_opt): # so i[0] is left and i[1] is right
    if left_sum == right_sum:
        left_sum += i[0][2]
        left_side_opt1.append(i[0])
        right_sum += i[1][2]
        right_side_opt1.append(i[1])
    else:
        if (i[0][0] == 'jaana'): # left, so switch to right
            right_sum += i[0][2]
            left_sum += i[1][2]
            right_side_opt1.append(i[0])
            left_side_opt1.append(i[1])
        elif (i[1][0] == 'jaana'): # right, so switch to left
            left_sum += i[1][2]
            right_sum += i[0][2]
            left_side_opt1.append(i[1])
            right_side_opt1.append(i[0])
        elif left_sum >= right_sum: # add smaller to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[1][2]
                left_side_opt1.append(i[1])
                right_sum += i[0][2]
                right_side_opt1.append(i[0])
            else: 
                left_sum += i[0][2]
                left_side_opt1.append(i[0])
                right_sum += i[1][2]
                right_side_opt1.append(i[1])
        else:  # add bigger to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[0][2]
                left_side_opt1.append(i[0])
                right_sum += i[1][2]
                right_side_opt1.append(i[1])
            else: 
                left_sum += i[1][2]
                left_side_opt1.append(i[1])
                right_sum += i[0][2]
                right_side_opt1.append(i[0])
                
# second sitting

to_write = pd.DataFrame([cox, [i[0] for i in left_side_opt1], [i[0] for i in right_side_opt1]])
to_write.index = ['cox', 'left_side', 'right_side']
to_write.to_csv('sitting2.csv', header=False)

In [42]:
# third sitting, 



temp = left_side_opt1[0] # left 0 becomes cox
left_side_opt1[0] = right_side_opt1[1] # 
right_side_opt1[1] = right_side_opt1[0]
right_side_opt1[0] = cox[0]
cox = [temp]

left_side_opt2, right_side_opt2 = [], []
left_sum, right_sum = 0,0

for i in zip(left_side_opt1, right_side_opt1): # so i[0] is left and i[1] is right
    if left_sum == right_sum:
        left_sum += i[0][2]
        left_side_opt2.append(i[0])
        right_sum += i[1][2]
        right_side_opt2.append(i[1])
    else:
        if (i[0][0] == 'jaana'): # left, so switch to right
            print(i)
            right_sum += i[0][2]
            left_sum += i[1][2]
            right_side_opt2.append(i[0])
            left_side_opt2.append(i[1])

        elif left_sum >= right_sum: # add smaller to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[1][2]
                left_side_opt2.append(i[1])
                right_sum += i[0][2]
                right_side_opt2.append(i[0])
            else: 
                left_sum += i[0][2]
                left_side_opt2.append(i[0])
                right_sum += i[1][2]
                right_side_opt2.append(i[1])
        else:  # add bigger to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[0][2]
                left_side_opt2.append(i[0])
                right_sum += i[1][2]
                right_side_opt2.append(i[1])
            else: 
                left_sum += i[1][2]
                left_side_opt2.append(i[1])
                right_sum += i[0][2]
                right_side_opt2.append(i[0])
                
# third sitting

to_write = pd.DataFrame([cox, [i[0] for i in left_side_opt2], [i[0] for i in right_side_opt2]])
to_write.index = ['cox', 'left_side', 'right_side']
to_write.to_csv('sitting3.csv', header=False)

In [43]:
# forth sitting


temp = left_side_opt2[0] # left 0 becomes cox
left_side_opt2[0] = right_side_opt2[0]
right_side_opt2[0] = cox[0]
cox = [temp]

left_side_opt3, right_side_opt3 = [], []
left_sum, right_sum = 0,0

for i in zip(left_side_opt2, right_side_opt2): # so i[0] is left and i[1] is right
    if left_sum == right_sum:
        left_sum += i[0][2]
        left_side_opt3.append(i[0])
        right_sum += i[1][2]
        right_side_opt3.append(i[1])
    else:
        if (i[0][0] == 'jaana'): # left, so switch to right
            print(i)
            right_sum += i[0][2]
            left_sum += i[1][2]
            right_side_opt3.append(i[0])
            left_side_opt3.append(i[1])

        elif left_sum >= right_sum: # add smaller to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[1][2]
                left_side_opt3.append(i[1])
                right_sum += i[0][2]
                right_side_opt3.append(i[0])
            else: 
                left_sum += i[0][2]
                left_side_opt3.append(i[0])
                right_sum += i[1][2]
                right_side_opt3.append(i[1])
        else:  # add bigger to left
            if i[0][2] >= i[1][2]: # left is bigger
                left_sum += i[0][2]
                left_side_opt3.append(i[0])
                right_sum += i[1][2]
                right_side_opt3.append(i[1])
            else: 
                left_sum += i[1][2]
                left_side_opt3.append(i[1])
                right_sum += i[0][2]
                right_side_opt3.append(i[0])
                
# third sitting

to_write = pd.DataFrame([cox, [i[0] for i in left_side_opt3], [i[0] for i in right_side_opt3]])
to_write.index = ['cox', 'left_side', 'right_side']
to_write.to_csv('sitting4.csv', header=False)

(['jaana', 11, 62.0], ['tatiana', 10, 55.5])


#### playground

In [55]:
boat = np.array((left,right))

In [58]:
boat

array([[55. , 79. , 55.1, 65.1, 55.5, 73. , 60. ],
       [70. , 67.5, 65. , 64.5, 62. , 64.1, 64. ]])

### we iterate pairwise over elements of the 2d array, and then minimize variance of sums of weights of each sides by checking (a,b) and (b,a)

In [60]:
np.nanvar(boat,axis=1)

array([78.80816327,  5.95428571])

In [62]:
var(boat[1])

5.954285714285716

### so the idea is to minimiza the difference between the rows, so pair-wise sum of rowers' weights in pos 2,4.. & 3,5.. has the smallest difference.

in other words it is known as an optimization version of the partition problem https://en.wikipedia.org/wiki/Partition_problem
https://stackoverflow.com/questions/38133748/how-to-swap-members-of-two-arrays-so-that-the-difference-of-sum-of-the-elements

One approach to the problem, imitating the way children choose teams for a game, is the greedy algorithm, which iterates through the numbers in descending order, assigning each of them to whichever subset has the smaller sum. This approach has a running time of O(n log n). This heuristic works well in practice when the numbers in the set are of about the same size as its cardinality or less, but it is not guaranteed to produce the best possible partition. For example, given the set S = {4, 5, 6, 7, 8} as input, this greedy algorithm would partition S into subsets {4, 5, 8} and {6, 7}; however, S has an exactly balanced partition into subsets {7, 8} and {4, 5, 6}.

```
def find_partition(int_list):
    "returns: An attempt at a partition of `int_list` into two sets of equal sum"
    A = set()
    B = set()
    for n in sorted(int_list, reverse=True):
        if sum(A) < sum(B):
           A.add(n)
        else:
           B.add(n)
    return (A, B)
```

In [332]:
print(np.var((left, right)))
print(left, right)

51.840000000000245
442.7 457.1


In [335]:
print(np.var((left+7, right-7)))
print(left, right)

0.04000000000000682
442.7 457.1


In [338]:
np.var((3,1))

1.0

In [339]:
a = np.array([[1, np.nan], [3, 4]])

In [343]:
np.nanvar(a, axis=1)

array([0.  , 0.25])

In [347]:
a = np.empty((5,))

In [348]:
a

array([0., 0., 0., 0., 0.])